# 🐝 TutorBee - Smart Business Agent

This notebook implements an AI-powered chatbot for TutorBee, an interactive tutoring service.

## Features:
- Answer questions about TutorBee's services
- Collect customer leads (name, email, notes)
- Record unanswered questions and feedback
- Deploy via Gradio interface

### Deployed HuggingFace link:
https://huggingface.co/spaces/safaasalman/tutorbee-assistant

## 1. Install Required Libraries

In [1]:
!pip install openai gradio python-dotenv PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00


## 2. Import Libraries

In [2]:
import os
import json
import gradio as gr
from openai import OpenAI
from datetime import datetime
from typing import Dict, List, Any
import PyPDF2
from google.colab import userdata

## 3. Load Business Information

**Note:** Upload your `about_business.pdf` and `business_summary.txt` files from the `me/` folder to Colab before running this cell.

In [9]:
def load_business_context():
    """
    Load business information from uploaded files.
    """
    context = ""

    # Load text summary
    try:
        with open('/content/me/business_summary.txt', 'r', encoding='utf-8') as f:
            context += f.read() + "\n\n"
        print("✅ Loaded business_summary.txt")
    except FileNotFoundError:
        print("⚠️ business_summary.txt not found. Please upload it.")

    # Load PDF content
    try:
        with open('/content/me/about_business.pdf', 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            pdf_text = ""
            for page in pdf_reader.pages:
                pdf_text += page.extract_text()
            context += "Additional Business Information:\n" + pdf_text
        print("✅ Loaded about_business.pdf")
    except FileNotFoundError:
        print("⚠️ about_business.pdf not found. Please upload it.")

    return context

# Load business context
BUSINESS_CONTEXT = load_business_context()
print("\n📚 Business context loaded successfully!")

✅ Loaded business_summary.txt
✅ Loaded about_business.pdf

📚 Business context loaded successfully!


## 4. Define Tool Functions

These functions will be called by the AI agent to collect leads and feedback.

In [10]:
# Storage for leads and feedback
leads_database = []
feedback_database = []

def record_customer_interest(email: str, name: str, message: str) -> str:
    """
    Record customer lead information.

    Args:
        email: Customer's email address
        name: Customer's name
        message: Additional notes or interests

    Returns:
        Confirmation message
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    lead_entry = {
        "timestamp": timestamp,
        "name": name,
        "email": email,
        "message": message
    }

    leads_database.append(lead_entry)

    # Log to console
    print("\n" + "="*50)
    print("📝 NEW LEAD RECORDED")
    print("="*50)
    print(f"Timestamp: {timestamp}")
    print(f"Name: {name}")
    print(f"Email: {email}")
    print(f"Message: {message}")
    print("="*50 + "\n")

    # Also save to file
    with open('leads_log.json', 'w') as f:
        json.dump(leads_database, f, indent=2)

    return f"Thank you, {name}! Your interest has been recorded. We'll contact you at {email} soon."


def record_feedback(question: str) -> str:
    """
    Record unanswered questions or customer feedback.

    Args:
        question: The question or feedback that couldn't be answered

    Returns:
        Confirmation message
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    feedback_entry = {
        "timestamp": timestamp,
        "question": question
    }

    feedback_database.append(feedback_entry)

    # Log to console
    print("\n" + "="*50)
    print("💬 FEEDBACK/UNANSWERED QUESTION RECORDED")
    print("="*50)
    print(f"Timestamp: {timestamp}")
    print(f"Question: {question}")
    print("="*50 + "\n")

    # Also save to file
    with open('feedback_log.json', 'w') as f:
        json.dump(feedback_database, f, indent=2)

    return "Thank you for your question. I've recorded it and our team will review it to improve our service."


print("✅ Tool functions defined successfully!")

✅ Tool functions defined successfully!


## 5. Define Tool Schemas for OpenAI Function Calling

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "record_customer_interest",
            "description": "Record a customer's contact information and interest in TutorBee services. Use this when a customer wants to sign up, get more information, or schedule a session.",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {
                        "type": "string",
                        "description": "The customer's email address"
                    },
                    "name": {
                        "type": "string",
                        "description": "The customer's full name"
                    },
                    "message": {
                        "type": "string",
                        "description": "Additional notes about what the customer is interested in or their specific needs"
                    }
                },
                "required": ["email", "name", "message"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": "Record questions that you cannot answer or customer feedback for future improvement. Use this when you don't have enough information to answer a customer's question.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The unanswered question or feedback from the customer"
                    }
                },
                "required": ["question"]
            }
        }
    }
]

print("✅ Tool schemas defined successfully!")

✅ Tool schemas defined successfully!


## 6. Set Up OpenAI API

**Important:** Add your OpenAI API key to Colab Secrets:
1. Click the key icon (🔑) in the left sidebar
2. Add a secret named `OPENAI_API_KEY`
3. Paste your API key as the value

In [12]:
# Get API key from Colab secrets
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("✅ OpenAI client initialized successfully!")
    print("📌 Using Model: gpt-4o-mini")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Please add your OPENAI_API_KEY to Colab Secrets.")

✅ OpenAI client initialized successfully!
📌 Using Model: gpt-4o-mini


## 7. Create the TutorBee Agent Class

### System Prompt Design Principles:

The system prompt is carefully crafted to:

✅ **Stay In Character**: Agent acts as "Alex from TutorBee", never breaking character

✅ **Use Business Context**: All answers come from the loaded PDF and TXT files - no hallucination

✅ **Log Unknown Questions**: Automatically calls `record_feedback()` for anything not in the documentation

✅ **Encourage Lead Collection**: Proactively asks for contact info when customers show interest

✅ **Professional & Friendly**: Balances warmth with professionalism to build trust

### 🔄 Agent Interaction Flow

The agent follows this interaction pattern:

```
1. User sends message
   ↓
2. Add message to chat history
   ↓
3. Send to OpenAI API with:
   - Chat history (context)
   - System prompt (behavior)
   - Available tools (functions)
   ↓
4. Model decides:
   - Answer directly? → Return text
   - Need to call tool? → Execute function
   ↓
5. If tool called:
   - Execute Python function
   - Add result to history
   - Get final response from model
   ↓
6. Return response to user
```

This implements the standard **function calling** pattern used by OpenAI.

In [13]:
class TutorBeeAgent:
    """
    Smart agent for TutorBee business interactions using OpenAI.

    This class implements the core agent interaction pattern:
    1. Receives user input
    2. Maintains chat history for context
    3. Sends request to AI model with available tools
    4. Handles tool calls if model requests them
    5. Returns final response to user
    """

    def __init__(self, client, business_context: str):
        """
        Initialize the TutorBee agent.

        Args:
            client: OpenAI client instance
            business_context: Business information loaded from PDF and TXT files
        """
        self.client = client
        self.business_context = business_context
        self.conversation_history = []

        # System prompt - defines agent behavior
        self.system_prompt = f"""You are Alex, a friendly and professional customer service representative for TutorBee, an AI-powered interactive tutoring service.

CRITICAL: You must ALWAYS stay in character as a TutorBee representative. Never break character or reveal that you are an AI language model.

Your role and responsibilities:
1. Answer questions about TutorBee using ONLY the information provided in the business documentation below
2. Help potential customers understand how TutorBee can transform their learning journey
3. Proactively encourage interested visitors to leave their contact information
4. Use the record_feedback tool for ANY question you cannot answer from the provided information

Business Information (THIS IS YOUR ONLY SOURCE OF TRUTH):
{self.business_context}

Behavioral Guidelines:
- Stay in character as "Alex from TutorBee" - you work here and are passionate about education
- Be warm, conversational, enthusiastic, and genuinely helpful
- Use the business information above to answer ALL questions - do not make up information
- If asked about pricing, features, policies, or anything NOT in the documentation, use record_feedback tool immediately
- Proactively ask for contact details when customers show ANY interest (questions about features, pricing, scheduling, etc.)

Lead Collection Strategy:
- When a customer asks multiple questions or shows sustained interest, say something like: "I'd love to help you get started! May I have your name and email so we can send you personalized information?"
- When someone asks about signing up, pricing, or scheduling: "Great! Let me collect your details so our team can reach out. What's your name and email?"
- Make it natural and helpful, not pushy

Unknown Question Protocol:
- If you don't know the answer from the business information provided, immediately say: "That's a great question! I don't have that specific information right now, but let me record this for our team to follow up with you."
- Then use the record_feedback tool to log it
- NEVER make up information not in the business documentation

Response Style:
- Keep responses concise (2-4 sentences typically)
- Use emojis sparingly to add warmth (🐝, 📚, ✨, 🎯)
- Be professional yet personable
- Show genuine excitement about helping students succeed
"""

        # Initialize chat history with system prompt
        self.conversation_history.append({
            "role": "system",
            "content": self.system_prompt
        })

    def chat(self, user_message: str) -> str:
        """
        Process user message and return agent response.

        This method implements the complete interaction flow:
        1. Add user message to chat history
        2. Send to AI model with tools available
        3. Check if model wants to call a tool
        4. If yes: execute tool and get final response
        5. If no: return generated text directly

        Args:
            user_message: The user's input text

        Returns:
            The agent's response text
        """
        # Step 1: Add user message to history
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })

        # Step 2: Call OpenAI API with function calling capability
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=self.conversation_history,
            tools=tools,  # Available functions
            tool_choice="auto"  # Let model decide when to use tools
        )

        assistant_message = response.choices[0].message

        # Step 3: Check if the model wants to call a function
        if assistant_message.tool_calls:
            # Model requested tool execution
            self.conversation_history.append(assistant_message)

            # Step 4: Process each tool call
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                # Execute the appropriate function
                if function_name == "record_customer_interest":
                    function_response = record_customer_interest(
                        email=function_args.get("email"),
                        name=function_args.get("name"),
                        message=function_args.get("message")
                    )
                elif function_name == "record_feedback":
                    function_response = record_feedback(
                        question=function_args.get("question")
                    )
                else:
                    function_response = "Function not found."

                # Add function response to conversation history
                self.conversation_history.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response
                })

            # Step 5: Get final response after function execution
            second_response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=self.conversation_history
            )

            final_message = second_response.choices[0].message.content
            self.conversation_history.append({
                "role": "assistant",
                "content": final_message
            })

            return final_message
        else:
            # No function call needed - return direct response
            response_text = assistant_message.content
            self.conversation_history.append({
                "role": "assistant",
                "content": response_text
            })
            return response_text

    def reset(self):
        """
        Reset conversation history to start fresh.
        Useful for testing or starting a new conversation.
        """
        self.conversation_history = [{
            "role": "system",
            "content": self.system_prompt
        }]

print("✅ TutorBeeAgent class created successfully!")
print("📌 Using OpenAI API with gpt-4o-mini model")

✅ TutorBeeAgent class created successfully!
📌 Using OpenAI API with gpt-4o-mini model


## 8. Create Gradio Interface

In [27]:
# Initialize the agent
agent = TutorBeeAgent(client, BUSINESS_CONTEXT)
print("🤖 TutorBee agent initialized and ready!")

def gradio_chat(message, history):
    """
    Wrapper function for Gradio chat interface.
    """
    response = agent.chat(message)
    return response

def view_leads():
    """
    Display collected leads.
    """
    if not leads_database:
        return "No leads collected yet."

    output = "## 📝 Collected Leads\n\n"
    for i, lead in enumerate(leads_database, 1):
        output += f"**Lead #{i}**\n"
        output += f"- **Time:** {lead['timestamp']}\n"
        output += f"- **Name:** {lead['name']}\n"
        output += f"- **Email:** {lead['email']}\n"
        output += f"- **Message:** {lead['message']}\n\n"
    return output

def view_feedback():
    """
    Display collected feedback.
    """
    if not feedback_database:
        return "No feedback recorded yet."

    output = "## 💬 Recorded Feedback\n\n"
    for i, feedback in enumerate(feedback_database, 1):
        output += f"**Feedback #{i}**\n"
        output += f"- **Time:** {feedback['timestamp']}\n"
        output += f"- **Question:** {feedback['question']}\n\n"
    return output

# Create Gradio interface
with gr.Blocks(title="TutorBee Assistant", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🐝 Welcome to TutorBee!
        ### Your AI-Powered Interactive Tutoring Service

        Ask me anything about our services, pricing, team, or how we can help you succeed!
        """
    )

    with gr.Tab("💬 Chat"):
        chatbot = gr.ChatInterface(
            fn=gradio_chat,
            examples=[
                "What services does TutorBee offer?",
                "Tell me about your team",
                "How does the AI tutoring assistant work?",
                "I'm interested in signing up for math tutoring",
                "What makes TutorBee different from other tutoring platforms?"
            ],
            title="Chat with TutorBee Assistant"
        )

    with gr.Tab("📊 Analytics"):
        gr.Markdown("## View Collected Data")

        with gr.Row():
            leads_btn = gr.Button("View Leads", variant="primary")
            feedback_btn = gr.Button("View Feedback", variant="secondary")

        output_display = gr.Markdown()

        leads_btn.click(fn=view_leads, outputs=output_display)
        feedback_btn.click(fn=view_feedback, outputs=output_display)

print("✅ Gradio interface created successfully!")

🤖 TutorBee agent initialized and ready!


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


✅ Gradio interface created successfully!


## 9. Test the System Prompt

Test the agent's behavior before launching Gradio:

In [16]:
# Test 1: Ask about services (should answer from business context)
print("🧪 TEST 1: Asking about services\n")
response = agent.chat("What services does TutorBee offer?")
print(f"Agent: {response}\n")
print("-" * 70)

🧪 TEST 1: Asking about services

Agent: TutorBee offers a range of interactive tutoring services, including:

1. **AI Tutoring Assistant** - Explains difficult topics in real time and adapts to each student’s learning pace.
2. **Smart Scheduling** - Automatically books tutoring sessions based on both tutor and student availability.
3. **Performance Tracking** - Summarizes lessons and provides progress insights after each session.
4. **Feedback Collection** - Allows students and tutors to leave feedback easily to continuously improve teaching quality.
5. **Multi-Subject Support** - Covers key subjects like Math, Science, Languages, and Programming.

If you're interested in how any specific service works, I'd love to provide more information! May I have your name and email to send personalized details?

----------------------------------------------------------------------


In [17]:
# Test 2: Ask unknown question (should trigger record_feedback)
print("\n🧪 TEST 2: Asking unknown question\n")
response = agent.chat("What's your refund policy?")
print(f"Agent: {response}\n")
print("-" * 70)


🧪 TEST 2: Asking unknown question


💬 FEEDBACK/UNANSWERED QUESTION RECORDED
Timestamp: 2025-10-19 16:57:50
Question: What's TutorBee's refund policy?

Agent: I've recorded your question about the refund policy, and our team will review it to provide better information. If you have any other questions or if you're interested in our services, feel free to ask! Also, may I have your name and email to send you updates?

----------------------------------------------------------------------


In [18]:
# Test 3: Express interest (should trigger lead collection)
print("\n🧪 TEST 3: Expressing interest\n")
response = agent.chat("I'm interested in math tutoring for my daughter. My name is Sarah Johnson and my email is sarah.j@email.com")
print(f"Agent: {response}\n")
print("-" * 70)


🧪 TEST 3: Expressing interest


📝 NEW LEAD RECORDED
Timestamp: 2025-10-19 16:57:53
Name: Sarah Johnson
Email: sarah.j@email.com
Message: Interested in math tutoring for daughter.

Agent: Thank you, Sarah Johnson! I've noted your interest in math tutoring for your daughter, and our team will reach out to you soon at your email, sarah.j@email.com. If you have any more questions or need further assistance in the meantime, feel free to ask! 📚✨

----------------------------------------------------------------------


In [19]:
# Reset agent for clean Gradio session
agent.reset()
print("\n✅ Agent reset - ready for Gradio deployment!")


✅ Agent reset - ready for Gradio deployment!


## 9. Launch the Application

In [ ]:
# Launch the Gradio app
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://464ae4737ea32f888f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 10. Testing & Utilities

In [21]:
# View all collected leads
print("\n📝 LEADS DATABASE:")
print(json.dumps(leads_database, indent=2))


📝 LEADS DATABASE:
[
  {
    "timestamp": "2025-10-19 16:57:53",
    "name": "Sarah Johnson",
    "email": "sarah.j@email.com",
    "message": "Interested in math tutoring for daughter."
  }
]


In [22]:
# View all collected feedback
print("\n💬 FEEDBACK DATABASE:")
print(json.dumps(feedback_database, indent=2))


💬 FEEDBACK DATABASE:
[
  {
    "timestamp": "2025-10-19 16:57:50",
    "question": "What's TutorBee's refund policy?"
  }
]


In [23]:
# Download leads as JSON file
from google.colab import files

if leads_database:
    files.download('leads_log.json')
    print("✅ Leads downloaded!")
else:
    print("No leads to download.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Leads downloaded!


In [24]:
# Download feedback as JSON file
from google.colab import files

if feedback_database:
    files.download('feedback_log.json')
    print("✅ Feedback downloaded!")
else:
    print("No feedback to download.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Feedback downloaded!
